# TP 3 : classification de documents

(inspiré pour partie d'un TP par Antoine Simoulin)

La **classification de textes** est la tâche qui prend en entrée du texte, et prédit en sortie une ou plusieurs classes pour ce texte, l'important étant que l'ensemble des classes possibles est connu et fixé a priori.

## Exemples de tâches relevant de la "classification de texte"

De nombreuses applications de TAL correspondent à de la classification de textes. Par exemple :
* L'"**analyse de sentiments**" (ou "sentiment analysis"): est un nom pompeux pour la tâche d'identification de  la polarité positive ou négative d'un texte, appliqué par exemple pour déterminer
  * ce client est-il content ou pas?
  * ce spectateur a-t-il aimé ce film ou pas?
  * ...
* La détection de Spam dans les Emails
* Le suivi de tendances sur les réseaux sociaux
* Recherche de réponse dans une FAQ...


## Les données 

On utilise les données de classification de textes pour le français, telles qu'inclusent dans les données FLUE <a href="https://aclanthology.org/2020.lrec-1.302/">(Le et al., 2020)</a>, cf. le repo https://github.com/getalp/Flaubert/tree/master/flue.

Nous utilisons ici un extrait de la partie française d'un corpus d'"analyse de sentiment" multilingue, le corpus CLS <a href="https://aclanthology.org/P10-1114/">(Prettenhofer and Stein, 2010)</a>.

Les textes sont des revues par des utilisateurs, issues du site Amazon pour trois catégories de produits (livres, DVD et musique). Initialement, chaque exemple contenait une revue associée à une note allant de 1 à 5, mais les revues avec une note de 3 ont été écartées, et les notes ont été binarisées: 1/2 devient 0 ("négatif"), et 4/5 devient 1 ("positif"). 

Pour chaque catégorie de produit (livres, dvd, musique), les données cls-fr de FLUE contiennent 2000 revues de test, et 2000 revues d'apprentissage. Pour ce TP, nous avons en outre divisé les 2000 revues d'apprentissage en 1600 pour l'entraînement, et 400 pour la validation.



### Import des librairies

In [1]:
%%capture

# ⚠️ Execute only if running in Colab
if 'google.colab' in str(get_ipython()):
  IN_COLAB = True
else:
  IN_COLAB = False

if IN_COLAB:
  !pip install -q scikit-learn==0.23.2 matplotlib==3.3.2 pandas==1.1.3 nltk==3.5 spacy==2.3.2 
  !python3 -m spacy download fr_core_news_md
  # if running Colab, restart after libraries installation (Redémarrer l'environnement d'exécution)
  # exit()

In [2]:
import os, sys
import numpy as np  # python base math library
import pandas as pd # data structure

from collections import Counter
#from pprint import pprint
#from time import time
import logging
import itertools
import matplotlib.pyplot as plt

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

# Inline Figures with matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
def load_cls_file(file_path):
    reviews, labels = [], []
    with open(file_path,encoding='utf-8') as f:
        line = f.readline()
        while line:
            review, label = line.strip().split('\t')
            reviews.append(review.strip("\""))
            labels.append(int(label))
            line = f.readline()
    return reviews, labels

def load_cls_dataset(file_path, section='books'):
    part2X, part2y = {}, {}
    for part in ['train', 'valid']:  
        part2X[part], part2y[part] = load_cls_file(os.path.join(file_path, section, part+'_0.tsv'))
    return part2X, part2y

## Chargement des données

In [4]:
data_dir = './data_flue_cls_fr/'

In [5]:
X = {} # key1=cat, key2=part, val= list of reviews
y = {} # key1=cat, key2=part, val= list of labels
y_train = [] # les labels "gold" pour la partie train
y_valid = [] # les labels "gold" pour la partie validation
for genre in ['books', 'dvd', 'music']:
    X[genre], y[genre] = load_cls_dataset(data_dir, genre)
    y_train += y[genre]['train']
    y_valid += y[genre]['valid']

In [6]:
df = {} # key=cat, val = data frame
for part in ['train', 'valid']:
    df[part] = pd.DataFrame.from_dict(
    {'review': X['books'][part] + X['dvd'][part] + X['music'][part],
     'label': y['books'][part] + y['dvd'][part] + y['music'][part],
     'genre': ['books' for _ in range(len(X['books'][part]))] \
             + ['dvd' for _ in range(len(X['dvd'][part]))] \
             + ['music' for _ in range(len(X['music'][part]))]})

id2label = ['Négatif', 'Positif']

In [7]:
pd.options.display.max_colwidth = 300
df['train'].head(10)

,review,label,genre
0,"Je voulais mettre 0 étoile mais c'est pas possible... Commençons par le positif (c'est rapide): ça parle de tout : muxle, endurance, souplesse, alimentation, échauffement mais ... mal, désespérement mal. L'auteur prétend avoir fait 15 ans de recherche avant de pondre son opus majus. Ben faudrait...",0,books
1,"Le récit de vie d'une femme américaine, avec ses contradictions, ses rêves de liberté, de justice (dans l'amérique de la guerre du Vietnam). Un récit qui n'est pas prêt de me quitter sur la complexité de la nature humaine dans ce qu'elle a de plus tortueux et de plus insaisissable. Un récit cons...",1,books
2,"Quel chef d'oeuvre que cette 'autobiographie' ! C'est à croire que la vie de Michaël Crichton est aussi palpitante, voire plus par moments, que celle des personnages de ses romans... Que d'aventures, de découvertes, d'expérience ! On comprend mieux, après la lecture de cet ouvrage, d'où lui vien...",1,books
3,"Si vous cherchez un livre simple d'initiation à Scheme pour comprendre et modifier les Script-Fu de Gimp, ne faites pas comme moi, choisissez-en un autre. Si vous voulez étudier ce langage d'un point de vue théorique, sans allumer votre ordinateur, en entrant dans le détail des algorithmes, sans...",1,books
4,Ce livre est très intéressant à lire à plus d'un titre. Certes JF Revel voit juste dans son analyse au sujet de la pensée ambiante vis-à-vis des USA. Cependant il manque totalement de rigueur scientifique (beaucoup d'assertions sans références bibliographiques). Cet ouvrage bien qu'éclairant ne ...,0,books
5,"J'avais déjà été déçue par """"Dossier Benton"""". Mais là, que dire de """"L'Ile des Chiens"""" ??? Quel ennui ! Les personnages sont burlesques, et l'intrigue initéressante ! Je suis pourtant une inconditionnelle de celle que je considérais comme LA reine du thriller. On ne ressuscitera pas Benton, ma...",0,books
6,"Michel Hoàng trace un portrait captivant du plus grand conquérant de l'histoire. Il dépeint les mœurs de ces nomades de la steppe qui engendreront Tèmudjin, le fils de petit chef qui s'élèvera au range de Khan océanique. L'ascension lente et patiente faite à coup d'alliances et « desalliance » d...",1,books
7,"Que lui avons-nous fait? Pourquoi la belle Amélie se croit-elle obligée de nous décevoir chaque année? Si le but est de nous prouver que le génie ne s'utilise pas comme une clé de 12, la démonstration en a été faite depuis longtemps.",0,books
8,"Il y a deux manières d'envisager ce livre. Sur le plan de la méthode, Encel expose le Golan d'un point de vue géopolitique, ce qui peut être pertinent... jusqu'à un certain point. En effet, au-delà de certaines imprécisions, voire confusions dans la réalité historique du Golan, l'auteur en vient...",0,books
9,"On s'attend avec ce livre à découvrir un pays, une époque, et surtout lire un roman d'espionnage, quelque chose de palpitant, qui s'emballe, un livre que l'on ne peut plus quitter quand on l'a ouvert. Malheureusement, le scénario s'essouffle au fil des pages, pour devenir carrément asthmatique à...",0,books


## Exploration des données

Avant de commencer à dérouler un cas d'usage, vous devez toujours **analyser les données**. Vérifiez les valeurs manquantes ou abérrantes, la distribution des variables, l'équilibre des classes, sélectionnez ou écartez les variables en fonction de leur pertinence ou selon des critères éthiques.

### TODO1: Comparer la distribution des labels positifs / négatifs, et des genres (music, dvd, books) entre le corpus train et le corpus de test

In [8]:
df['train'].columns

Index(['review', 'label', 'genre'], dtype='object')

## Pré-traitements des textes: tokenisation, lemmatisation

### TODO2 : normalisation du vocabulaire

- tokenisez et lemmatisez les revues en utilisant Spacy (cf. TP2)
  * NB: dans le pipeline spacy, ci-dessous, on ne conserve que les modules nécessaires à la lemmatisation
- vous **stockerez** ces informations dans la DataFrame, pour chaque revue, les champs "tokens" et "lemmas"

- **comparez** les tailles de vocabulaire, pour les revues du "train", pour les 2 types de tokens
  -- les tokens obtenus après tokenization spacy
  -- les lemmes obtenus après lemmatisation spacy


In [46]:
import spacy

try:
    #rem: enable does not seem to work, don't know why
    nlp = spacy.load("fr_core_news_md") #, enable=["tok2vec", "morphologizer", "lemmatizer"])
except OSError:
    !python -m spacy download fr_core_news_md
    nlp = spacy.load("fr_core_news_md") #, enable=["tok2vec", "morphologizer", "lemmatizer"])

# we won't need ner, parser, attribute_ruler
# NB: tok2vec and morphologizer seem to be necessary for lemmatization
nlp.remove_pipe("ner")
nlp.remove_pipe("parser")
nlp.remove_pipe("attribute_ruler")
print('Current pipeline:\n  '+'\n  '.join(nlp.pipe_names))



Current pipeline:
  tok2vec
  morphologizer
  lemmatizer


In [47]:
# remarque: le type de lemmatiseur Spacy, pour le modèle fr_core_news_md est "rule" = "par règles"
for c in nlp.components:
    print(c)
    if c[0] == 'lemmatizer':
        print(c[1].mode)

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fa6417a6f40>)
('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7fa6417a6e20>)
('senter', <spacy.pipeline.senter.SentenceRecognizer object at 0x7fa6417a64c0>)
('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer object at 0x7fa62f019780>)
rule


In [48]:
sample_text = """Je voulais mettre 0 étoile mais c'est pas possible...
 Commençons par le positif (c'est rapide): ça parle de tout : 
muxle, endurance, souplesse, alimentation, échauffement mais ... mal, désespérement mal."""

# testez la tokenization / lemmatisation sur le texte supra
# puis appliquez à l'ensemble des revues



In [50]:
%%time
# application aux revues de la lemmatisation Spacy et la racinisation NLTK
# NB: il est important de n'appeler qu'une seule fois le traitement spacy sur un texte
#     cf. coûteux en temps



CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [51]:
df['train'].head()

,review,label,genre
0,"Je voulais mettre 0 étoile mais c'est pas possible... Commençons par le positif (c'est rapide): ça parle de tout : muxle, endurance, souplesse, alimentation, échauffement mais ... mal, désespérement mal. L'auteur prétend avoir fait 15 ans de recherche avant de pondre son opus majus. Ben faudrait...",0,books
1,"Le récit de vie d'une femme américaine, avec ses contradictions, ses rêves de liberté, de justice (dans l'amérique de la guerre du Vietnam). Un récit qui n'est pas prêt de me quitter sur la complexité de la nature humaine dans ce qu'elle a de plus tortueux et de plus insaisissable. Un récit cons...",1,books
2,"Quel chef d'oeuvre que cette 'autobiographie' ! C'est à croire que la vie de Michaël Crichton est aussi palpitante, voire plus par moments, que celle des personnages de ses romans... Que d'aventures, de découvertes, d'expérience ! On comprend mieux, après la lecture de cet ouvrage, d'où lui vien...",1,books
3,"Si vous cherchez un livre simple d'initiation à Scheme pour comprendre et modifier les Script-Fu de Gimp, ne faites pas comme moi, choisissez-en un autre. Si vous voulez étudier ce langage d'un point de vue théorique, sans allumer votre ordinateur, en entrant dans le détail des algorithmes, sans...",1,books
4,Ce livre est très intéressant à lire à plus d'un titre. Certes JF Revel voit juste dans son analyse au sujet de la pensée ambiante vis-à-vis des USA. Cependant il manque totalement de rigueur scientifique (beaucoup d'assertions sans références bibliographiques). Cet ouvrage bien qu'éclairant ne ...,0,books


In [52]:
# comparaison des tailles de vocabulaires obtenues


## Représentation vectorielle de chaque revue: encodage "bag-of-word" (BOW) 

En classification automatique, les "objets" à classer doivent être représentés sous forme de vecteur, qui constitue l'entrée du classifieur.
En classif de documents, ces objets sont des documents.
Et on a vu en cours la représentation vectorielle la plus basique : Bag-of-Words (BOW).

**Rappel: Bag of Words** : On attribue un indice à chacun des mots du vocabulaire défini par le corpus d'entrainement. On peut ensuite représenter chaque document par un vecteur X indiquant, à la composante i le nombre d'occurrences du i-ème mot du vocabulaire. La taille du vocabulaire est généralement comprise entre 30.000 et 100.000 mots. Un vecteur BOW a bcp de valeurs nulles, cf. un document ne couvre qu'une toute petite partie du vocabulaire: un vecteur BOW est **creux** ("sparse" en anglais) et de grande taille.

In [53]:
# On considère ce corpus "jouet", constitué de 3 documents déjà tokenisés,
# i.e. chaque document est une liste de formes.

train_corpus = [
    ["je", "n'", "aime", "pas", "ce", "livre"],
    ["un", "livre", "très", "complet", ",", "un", "livre", "magique"],
    ["pas", "un", "livre", "exceptionnel", ",", "ni", "un", "livre", "très", "complet"]]

test_corpus = [
    ["un", "nouveau", "livre", "super"],
    ["et", "un", "autre", "livre", "magique"]
]

### TODO3: représentation BOW d'après un vocabulaire

La première chose à faire est d'associer chaque mot du vocabulaire à un identifiant entier. On part en général de 0. L'ordre n'a pas d'importance.

En général, le vocabulaire est défini comme tous les mots rencontrés dans un certain ensemble de documents (en général, l'ensemble d'apprentissage, train)).

* écrire une fonction **get_vocab** qui parcourt un ens. de documents tokenises (une liste de liste de tokens) et en ressort
  * un dictionnaire **w2id** : clé = mot, valeur = id du mot
    * pour aller des mots vers leur id
  * une liste **id2w**, où au rang i dans la liste, se trouve le mot d'identifiant i
    * pour récupérer d'un id vers le mot auquel il correspond
* écrire une fonction bow qui rend le vecteur bow d'un document (sous forme de liste, où le rang correspond à l'id d'un mot du vocabulaire)
* appliquez vos fonctions pour afficher les vecteurs BOW obtenus pour le mini corpus train_corpus et test_corpus
  * NB: le vocabulaire utilisé est celui du train_corpus
  * les mots présents dans le test mais absents du test_corpus (les "inconnus") seront ici simplement ignorés
    
  


In [54]:


def get_vocab(documents):
    w2id = {}
    id2w = []
    # TODO
    return (w2id, id2w)

def bow(doc, w2id, id2w):
    # TODO
    return bow_vector
    
# TODO : affichage des vecteurs BOW du corpus train, et du corpus test

print("BOW vectors of train_corpus")
    
print("BOW vectors of test_corpus (unknown words are ignored)")


BOW vectors of train_corpus
BOW vectors of test_corpus (unknown words are ignored)


### Comparaison avec les Vectorizer de sklearn

On peut utiliser la librairie sklearn pour comparer

On donne le code ci-dessous, étudiez-le.

In [55]:

from sklearn.feature_extraction.text import CountVectorizer

# ici notre corpus de départ (train_corpus) est déjà tokenisé,
# donc on déclare le vectorizer avec un tokenizer et preprocessor qui ne fait rien
def dummy(document):
    return document

vectorizer = CountVectorizer(tokenizer=dummy, preprocessor=dummy)

# au départ, le vectorizer est vide : donc ceci génère une erreur
#print(vectorizer.vocabulary_)
#print(vectorizer.get_feature_names())

# fit_transform va créer
# - le mapping entre id et mots (le vocabulaire)
# - et les vecteurs BOW de chaque document du train_corpus
#   sous la forme d'une matrice
X_train = vectorizer.fit_transform(train_corpus)

# la sortie est une matrice T x |V|
# - nb lignes = nb documents dans le train
# - nb colonnes = taille du vocabulaire
# C'est une matrice creuse, le type utilisé est scipy.sparse
print("type of X_train", type(X_train))
print("shape of X_train", X_train.shape)
print(X_train)

# on peut l'afficher de manière plus lisible
print(X_train.toarray()) 


type of X_train <class 'scipy.sparse.csr.csr_matrix'>
shape of X_train (3, 13)
  (0, 5)	1
  (0, 8)	1
  (0, 1)	1
  (0, 10)	1
  (0, 2)	1
  (0, 6)	1
  (1, 6)	2
  (1, 12)	2
  (1, 11)	1
  (1, 3)	1
  (1, 0)	1
  (1, 7)	1
  (2, 10)	1
  (2, 6)	2
  (2, 12)	2
  (2, 11)	1
  (2, 3)	1
  (2, 0)	1
  (2, 4)	1
  (2, 9)	1
[[0 1 1 0 0 1 1 0 1 0 1 0 0]
 [1 0 0 1 0 0 2 1 0 0 0 1 2]
 [1 0 0 1 1 0 2 0 0 1 1 1 2]]


In [56]:
# voici la correspondance entre id et mots (notre w2id supra)
print(vectorizer.vocabulary_)
# et la liste des mots (notre id2w)
print(vectorizer.get_feature_names_out())

{'je': 5, "n'": 8, 'aime': 1, 'pas': 10, 'ce': 2, 'livre': 6, 'un': 12, 'très': 11, 'complet': 3, ',': 0, 'magique': 7, 'exceptionnel': 4, 'ni': 9}
[',' 'aime' 'ce' 'complet' 'exceptionnel' 'je' 'livre' 'magique' "n'" 'ni'
 'pas' 'très' 'un']


In [57]:
# pour les documents de test:
# on ignore les nouveaux mots (qui sont "inconnus" dans le train)
# => on utilise la méthode "transform" au lieu de fit_transform

X_test = vectorizer.transform(test_corpus)
print("shape of X_test", X_test.shape)

# On peut voir que la taille du vocabulaire est bien constante,
# les mots inconnus dans le train ont simplement été ignorés
print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())



shape of X_test (2, 13)
{'je': 5, "n'": 8, 'aime': 1, 'pas': 10, 'ce': 2, 'livre': 6, 'un': 12, 'très': 11, 'complet': 3, ',': 0, 'magique': 7, 'exceptionnel': 4, 'ni': 9}
[',' 'aime' 'ce' 'complet' 'exceptionnel' 'je' 'livre' 'magique' "n'" 'ni'
 'pas' 'très' 'un']


## Vecteurs de documents, avec poids TF-IDF

cf. cours

$$\text{tf-idf}(t, d, D) = \text{tf}(t, d) \times \text{idf}(t, D)$$

où 
* `tf` est le **nombre d'occurrences du terme** t dans le document d
  * ou bien des variantes du simple nb d'occurrences 
* `idf` est la **inverse document frequency** du terme t dans l'ensemble de documents D
  * si on note $\mathrm{df}(t,D)$ = le nb de documents dans D dans lesquels le terme t apparaît
  * `sklearn` utilise une définition différente de l'IDF classique: 
$$  \mathrm{idf}(t, D) = 1 + \log \left( \frac{|D|}{\mathrm{df}(t, D)}\right) $$

  * et dans le cas ou `smooth_idf = True` $$\mathrm{idf}(t, D) = 1+ \log \left( \frac{1+|D|}{1+\mathrm{df}(t, D)}\right)   $$
  
### TODO4: le TfidfVectorizer de sklearn

**Utilisez** le TfidfVectorizer de sklearn (au lieu de CountVectorizer supra) pour obtenir les vecteurs BOW avec valeurs TF.IDF

Le vocabulaire sous-jacent à un corpus peut grossir très rapidement
(cf. loi de Zipf!), ce qui augmente donc directement 
- la taille de l'espace vectoriel de représentation des documents (= le nombre de "features" d'entrée)
- et le nombre de paramètres des classifieurs appris sur ces représentations
- cela peut amener du surapprentissage (overfitting) et/ou un apprentissage moins performant

Plusieurs options permettent de filtrer le vocabulaire pris en compte, en ignorant des termes soit trop peu discriminants, soit trop rares etc...

**Trouvez** dans la doc quelles options permettent de limiter la taille des vecteurs BOW résultants, en filtrant selon le nb d'occurrences d'un terme ou le nombre de documents dans lesquels apparaît un terme 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = ...
X_train_tfidf = ...
X_test_sk = ...



## Classifieur via sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:

vectorizer = CountVectorizer(tokenizer=dummy, 
                            preprocessor=dummy)
X_train = vectorizer.fit_transform(df['train']['tokens'])
X_valid = vectorizer.transform(df['valid']['tokens'])


Y_train = df['train']['label']
Y_valid = df['valid']['label']


In [ ]:
# instance de classifieur de type "régression logistique"

clf_f = LogisticRegression(
        #random_state=0, 
        solver='lbfgs',    # algo d'optimisation (ici minimisation de la perte cross-entropie)
        multi_class='ovr', # stratégie "one versus rest"
        penalty='l2', # hyperparamètre de régularisation
        C=1.0, # hyperparamètre de régularisation
        n_jobs=2
)

### Apprentissage

In [ ]:
# ----------- apprentissage sur le train -----------
# La méthode fit
# (vaut pour tous les types de classifieurs ou régresseurs !)
clf_f.fit(X_train, Y_train)

### Prédiction et évaluation

Dans le cas d'un classifieur "mono-label" (i.e. on demande une et une seule classe par revue), la métrique d'évaluation est simplement la **proportion** de revues bien classées par le système.

En anglais on parle d'**accuracy**, en français **précision** ou **exactitude**.

En outre une **matrice de confusion** permet de représenter quelles classes sont trop/pas assez prédites. Dans le cas de classification binaire, on parle de:
- vrais positifs: items de classe 1 prédits 1
- faux positifs: items de classe 0 prédits 1
- vrais négatifs: items de classe 0 prédits 0
- faux négatifs: items de classe 1 prédits 0

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:

# --------- prediction sur le test---------------
Y_valid_pred_f = clf_f.predict(X_valid_f)
print("SCORE OF LOGISTIC REGRESSION ON VALID: %.3f" % accuracy_score(Y_valid, Y_valid_pred))
    

# --------- prediction sur le train--------------
Y_train_pred_f = clf_f.predict(X_train_f)
print("SCORE OF LOGISTIC REGRESSION ON TRAIN: %.3f" % accuracy_score(Y_train, Y_train_pred))


In [ ]:
# Matrice de confusion

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matrice de confusion normalisée")
    else:
        print("Matrice de confusion, sans normalisation")

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiquette réelle')
    plt.xlabel('Etiquette prédite')

In [ ]:
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(confusion_matrix(Y_valid, Y_valid_pred_f), 
                      classes=id2label, 
                      title='Matrice de confusion, sans normalisation')
plt.show()


In [ ]:
# on teste si une pondération TFIDF améliore les résultats
clf_t = LogisticRegression(
        #random_state=0, 
        solver='lbfgs',    # algo d'optimisation (ici minimisation de la perte cross-entropie)
        multi_class='ovr', # stratégie "one versus rest"
        penalty='l2', # hyperparamètre de régularisation
        C=1.0, # hyperparamètre de régularisation
        n_jobs=2
)

clf_t.fit(X_train_t, Y_train)

# --------- prediction sur le test---------------
Y_valid_pred_t = clf_t.predict(X_valid_t)
print("SCORE OF LOGISTIC REGRESSION ON VALID (tfidf): %.3f" % accuracy_score(Y_valid, Y_valid_pred))
    

# --------- prediction sur le train--------------
Y_train_pred_t = clf_t.predict(X_train_t)
print("SCORE OF LOGISTIC REGRESSION ON TRAIN (tfidf): %.3f" % accuracy_score(Y_train, Y_train_pred))

#@@ => on peut voir que le tfidf n'a pas un gros impact sur les résultats

## Un autre type de classifieur linéaire : les SVM

### TODO6: Appliquez l'apprentissage, la prédiction et l'évaluation, en utilisant cette fois un classifieur de type **SVM** (**support vector machine**), avec les hyperparamètres par défaut
- cf. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
- Comparez **avec et sans utilisation de Tfidf** (CountVectorizer versus TfidfVectorizer).

**NB** : en apprentissage automatique, on distingue:
* les **paramètres**, qui sont des variables dont les valeurs sont fixées grâce au processus d'apprentissage. Celui-ci est en général un problème d'optimisation, résolu
  * soit de manière 
  * soit de manière approchée et itérative, auquel cas, en général, les paramètres sont initialisés (au hasard ou avec des valeurs pré-apprises sur d'autres tâches) et ajustés itérativement
* des **hyperparamètres**, qui sont des valeurs fixées en amont du processus d'apprentissage
  * chaque algo d'apprentissage va de paire avec un certain nombre d'hyperparamètres à choisir avant l'apprentissage
  * par exemple, pour un apprentissage de type régression logistique, on peut choisir d'inclure ou pas un terme régularisateur à la fonction de perte (cf. supra l'option penalty='l2' pour l'instance de LogisticRegression)
  * et en amont, les divers pré-traitements constituent des hyperparamètres
    * utilisation des formes fléchies, des lemmes ou des stems
    * pondération tf.idf ou pas
    * minusculisation ou pas
    * suppression ou pas des accents
    * si tf.idf, utilisation diverse des options max_df, min_df, max_features
    * etc...


In [ ]:
from sklearn import svm

# TODO 
# apprentissage SVM
# **avec et sans utilisation de Tfidf** (CountVectorizer versus TfidfVectorizer)
# prédiction
# évaluation sur corpus "valid" et sur corpus "train" (et affichage)


## Le réglage des hyperparamètres ("tuning hyperparameters")

Certains algorithmes vont avoir des performances très différentes selon les valeurs des hyperparamètres. Rechercher de bonnes valeurs d'hyperparamètres se dit **régler les hyperparamètres** (en anglais **tuning hyperparameters**).

Cela reste très expérimental, et fastidieux en particulier parce qu'on ne peut pas isoler les hyperparamètres les uns des autres: la meilleure valeur trouvée pour A, avec B=b, ne sera pas forcément la meilleure valeur pour A avec B=b'.

On en est simplement réduit à tester plusieurs combinaisons d'hyperparamètres, et choisir la meilleure sur ces tests. Plusieurs stratégies existent pour définir les combinaisons à tester:
* par tatonnement
  * éventuellement ok pour commencer à avoir une idée de l'ordre de grandeur des valeurs fonctionnant bien
  * mais il est recommandé d'avoir ensuite une approche plus systématique
* avec une **recherche en grille** (**grid search**): on teste de manière systématique les combinaisons de valeurs
  * par exemple on teste toutes les combinaisons avec A prenant les valeurs a, a'', a''' et B prenant les valeurs b, b' => 6 combinaisons.
  * mais le nb de combinaisons à tester peut devenir rapidement trop important
* ou encore avec une recherche aléatoire de combinaisons


### La validation croisée ("cross-validation")


**NB**: l'évaluation des performances pour chaque combinaison d'hyperparamètres doit être faite **sur des exemples non utilisés à l'apprentissage**. 
Une solution serait d'utiliser le corpus valid, mais cela biaiserait les résultats: la meilleure combinaison d'hyperparamètres obtenue serait celle valable pour le corpus valid. On veut garder celui-ci pour une évaluation finale.

C'est pourquoi on utilise plutôt la **validation croisée** (**cross-validation**) pour évaluer la performances de chaque combinaison d'hyperparamètes.

### BONUS TODO 7: explorez GridSearchCV et lancer une recherche en grille avec la grille d'hyperparamètres fournies infra.


In [ ]:
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedKFold

# définition des combinaisons d'hyperparamètres à tester
# comme l'existence de certains hyperparamètres dépend de la valeur d'autres hyperparamètre,
# ces combinaisons sont définies comme des dictionnaires
param_grid = [
    {'C': [0.001, 0.1, 1, 10, 100], 
     'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100], 
     'gamma': ['scale','auto', 0.001], # gamma n'est pertinent que si kernel=rbf
     'kernel': ['rbf']},
    ]

# on utilise un vectorizer trouvé en faisant varier diverses options
vectorizer = TfidfVectorizer(tokenizer=dummy, preprocessor=dummy,
                                            max_features = 10000,
                                            min_df = 2,
                                            max_df = 0.7)
X_train_f = vectorizer.fit_transform(df['train']['lemmas'])
X_valid_f = vectorizer.transform(df['valid']['lemmas'])

#TODO BONUS grid search
